<a href="https://colab.research.google.com/github/Heity94/TWSM_Lab/blob/main/Project/Notebooks/Ontology_Glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

#set path to data in Google Drive
data_path = "/content/drive/MyDrive/2022_Analytics Lab Student Projects/Data/Topic 1/Data_Team1/SynDet"

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import torch
#import duckdb
#import sqlalchemy

#!pip install -U sentence-transformers
#from sentence_transformers import util

In [3]:
nounphrases = pd.read_csv(data_path+"/SynDet_entities_train.csv")
nounphrases.head(10)

,Unnamed: 0,article_id,sentence_id,ent_id,noun_phrases,true_ent_id
0,0,188,188_3753_3772,videotex,which,0
1,1,188,188_3753_3772,videotex,we,0
2,2,188,188_3753_3772,videotex,an information,0
3,3,188,188_3753_3772,videotex,communication resource,0
4,4,188,188_3753_3772,videotex,the Minitel system,0
5,5,188,188_3828_3848,competitive advantage,It,0
6,6,188,188_3828_3848,competitive advantage,any rm,0
7,7,188,188_3828_3848,competitive advantage,the eld,0
8,8,188,188_3828_3848,competitive advantage,so competitive advantages,0
9,9,188,188_3848_3883,government,This,0


In [4]:
# Filter for article 188
nounphrases_188 = nounphrases.loc[nounphrases["article_id"]==188]

In [5]:
nounphrases_188['noun_phrases'] = nounphrases_188['noun_phrases'].astype(str)
print (nounphrases_188.dtypes)

Unnamed: 0       int64
article_id       int64
sentence_id     object
ent_id          object
noun_phrases    object
true_ent_id     object
dtype: object


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [6]:
ontology_syn = pd.read_csv(data_path+"/SynDet_ontology_train.csv", index_col=0)
ontology_syn.shape

(340613, 4)

In [7]:
# Check number of synonyms per entity_id
ent_sy_group = ontology_syn.groupby("entity_id")[["synonym"]].nunique().sort_values(by="synonym", ascending=False)
ent_sy_group

,synonym
entity_id,
individual participant,195173
survey,36130
three validities,13677
online learning system,8583
communication service infrastructure,7635
...,...
Hardy-Weinberg equilibrium testing,1
knowledge creation,1
commercial bank,1


In [8]:
# Downsample entities with more than 100 synonyms to a 100 synoyms (but always include "main entity_id" in subsample)
idx = ent_sy_group[ent_sy_group["synonym"]>100].index.tolist()

dfs = []
for entity in idx:
  t1 = ontology_syn[(ontology_syn.entity_id==entity)&(ontology_syn.synonym==entity)]
  t2 = ontology_syn[(ontology_syn.entity_id==entity)&(ontology_syn.synonym!=entity)].sample(99)
  dfs.append(pd.concat([t1,t2]))
dfs.append(ontology_syn.loc[~ontology_syn["entity_id"].isin(idx)]) # Append dataframe for all entities with less then 100 synonyms to list

# Combine all dataframes 
ontology_syn_ds = pd.concat(dfs).reset_index(drop=True)

In [9]:
#Show synonyms in ontology which appear in more than 1 entity
ontology_syn_ds[ontology_syn_ds.synonym.duplicated(keep=False)].sort_values(by="synonym")

,entity_id,category,label,synonym
22253,Pearson's Chi square test,methodological entity,ANALYSIS_METHOD,Chi2 test for independence
21871,Pearson's chi squared test of independence bet...,methodological entity,ANALYSIS_METHOD,Chi2 test for independence
22248,Pearson's Chi square test,methodological entity,ANALYSIS_METHOD,Chi2 test for independences
21865,Pearson's chi squared test of independence bet...,methodological entity,ANALYSIS_METHOD,Chi2 test for independences
26033,systems operations,domain specific entity,TOPIC,EDP operation
...,...,...,...,...
3966,procurement information system,domain specific entity,TECHNOLOGY,web procurements
29501,website,domain specific entity,TECHNOLOGY,web sites
29876,web application,domain specific entity,TECHNOLOGY,web sites
9832,Chi square metric,methodological entity,METRIC,χ 2S


In [10]:
#List of unique entities from ontology with synonyms with downsampling

ontology_syn_ds_unique = ontology_syn_ds.drop_duplicates(subset="synonym")

# Using Glove

## Create embedding for the noun phrases

In [11]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')


def create_corpus_tk(df):
    corpus = []
    for text in df:
        words = text.split()
        corpus.append(words)
    return corpus

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [12]:
# Create a corpus, where each noun phrase is splited in individual words 
corpus_nounphrase = create_corpus_tk(nounphrases_188["noun_phrases"])

corpus_nounphrase[2]

['an', 'information']

In [13]:
num_words_nounphrase = len(corpus_nounphrase)
num_words_nounphrase

3227

In [14]:
# Tokenizing the words

from keras.preprocessing.text import Tokenizer

tokenizer_nounphrase = Tokenizer(num_words=num_words_nounphrase)
tokenizer_nounphrase.fit_on_texts(nounphrases_188["noun_phrases"])

nounphrase_sequences = tokenizer_nounphrase.texts_to_sequences(nounphrases_188["noun_phrases"])
nounphrase_sequences[2]

[21, 3]

In [15]:
len(nounphrase_sequences)

3227

In [16]:
len(nounphrases_188["noun_phrases"])

3227

In [17]:
print(nounphrases_188["noun_phrases"][0])
print(nounphrase_sequences[0])

which
[26]


In [18]:
word_index_nounphrase = tokenizer_nounphrase.word_index
print("Number of unique words:", len(word_index_nounphrase))

Number of unique words: 822


In [19]:
# Create a dictionary of word embedding from Glove (6B tokens, 100D word vector)

embedding_dict = {}
model = "/content/drive/MyDrive/2022_Analytics Lab Student Projects/Data/Topic 1/Data_Team1/glove.6B.100d.txt"

with open(model, "r") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], "float32")
        embedding_dict[word] = vectors
f.close()

In [20]:
# To assign the pre-trained Glove embedding to our word index (individual word)

def embedding_word(word_index):
  num_words = len(word_index) + 1
  embedding = np.zeros((num_words, 100))
  for word, i in word_index.items():
    if i < num_words:
        emb_vec = embedding_dict.get(word)
        if emb_vec is not None:
            embedding[i] = emb_vec
  return embedding

In [21]:
embedding_nounphrase_word = embedding_word(word_index_nounphrase)

In [22]:
word_index_nounphrase["information"]

3

In [23]:
(embedding_nounphrase_word[3] == embedding_dict.get("information")).all()

True

In [24]:
# To create embedding for the nounphrases, where we get the average of the embeddings of all the individual words in that nounphrase 

def embedding(sequences, embedding_word):
  emb_list = []
  for v in sequences:
    v_list = []
    for index in v:
      v_list.append(embedding_word[index])
    emb_list.append(np.mean(v_list, axis=0))
  return emb_list

In [25]:
embedding_nounphrase = embedding(nounphrase_sequences, embedding_nounphrase_word)
len(embedding_nounphrase)

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


3227

In [26]:
len(nounphrase_sequences)

3227

## Creating embedding for the entities

In [27]:
corpus_entity = create_corpus_tk(ontology_syn_ds_unique["synonym"])

corpus_entity[0]

['individual', 'participant']

In [28]:
num_words_entity = len(corpus_entity)
num_words_entity

32373

In [29]:
tokenizer_entity = Tokenizer(num_words=num_words_entity)
tokenizer_entity.fit_on_texts(ontology_syn_ds_unique["synonym"])

entity_sequences = tokenizer_entity.texts_to_sequences(ontology_syn_ds_unique["synonym"])

In [30]:
word_index_entity = tokenizer_entity.word_index
print("Number of unique words:", len(word_index_entity))

Number of unique words: 5463


In [31]:
embedding_entity_word = embedding_word(word_index_entity)

In [32]:
embedding_entity = embedding(entity_sequences, embedding_entity_word)

# Compute the cosine similarity

In [33]:
from numpy.linalg import norm

In [34]:

def cosine_similarity(embedding_nounphrase, embedding_entity): 
  df = pd.DataFrame(columns=['cosine', 'nounphrase_id', 'entity_id'])
  for i1, np in enumerate(embedding_nounphrase): 
    df2 = pd.DataFrame(columns=['cosine', 'nounphrase_id', 'entity_id'])
    for i2, e in enumerate(embedding_entity): 
      cosine = np.dot(np,e)/(norm(np)*norm(e))
      a = pd.Series([cosine, i1, i2], index=df2.columns)
      df2 = df2.append(a, ignore_index=True)
    df2 = df2.sort_values(by=['cosine'], ascending=False)
    max = df2[1]
    df.append(max, ignore_index=True)
  return df

In [35]:
test_np = embedding_nounphrase[:1]
test_entity = embedding_entity[:10]
len(test_np)

1

In [37]:
cosine = np.dot(embedding_nounphrase[40], embedding_entity[3])/(norm(embedding_nounphrase[40])*norm(embedding_entity[3]))
cosine

0.7044787226025018

In [36]:
from sklearn.metrics.pairwise import cosine_similarity

df2 = pd.DataFrame(columns=['cosine', 'nounphrase_id', 'entity_id'])
cosine = np.dot(embedding_nounphrase[40],embedding_entity[3])/(norm(embedding_nounphrase[40])*norm(embedding_entity[3]))
#cosine = cosine_similarity(embedding_nounphrase[40], embedding_entity[3])

a = pd.Series([cosine, 40, 3], index=df2.columns)
df2 = df2.append(a, ignore_index=True)
df2

,cosine,nounphrase_id,entity_id
0,0.704479,40.0,3.0


In [38]:
for i1, np in enumerate(test_np): 
    df2 = pd.DataFrame(columns=['cosine', 'nounphrase_id', 'entity_id'])
    for i2, e in enumerate(test_entity): 
      cosine = np.dot(np,e)/(norm(np)*norm(e))
      a = pd.Series([cosine, i1, i2], index=df2.columns)
      df2 = df2.append(a, ignore_index=True)
    df2 = df2.sort_values(by=['cosine'], ascending=False)
    max = df2[1]
    print(max)

ValueError: ignored

In [ ]:
test_cosine = cosine_similarity(test_np, test_entity)
test_cosine

NameError: ignored

In [ ]:
nounphrases_188["noun_phrases"][40]

'a market-led strategy'

In [ ]:
corpus_entity[3]

['research', 'and', 'executive', 'multicultural', 'professional']

# Test

In [ ]:
# Test 
!pip install -U sentence-transformers

from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
embeddings_sentences = model.encode(nounphrases_188.noun_phrases.to_list(), convert_to_tensor=True, show_progress_bar=True)

embeddings_sentences.shape

In [ ]:
embeddings_sentences.shape

torch.Size([3206, 384])